## Inference using Deeplabv3+ V6 on full Mapillary dataset

In [3]:
import os
import random
import numpy as np
import pandas as pd
from PIL import Image
import torch
import torchvision.transforms as T
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from torchvision import models
import segmentation_models_pytorch as smp
from PIL import ImageDraw

In [ ]:
#Configuration
device = torch.device("cuda")
model_path = "../best_deeplabmodels/V6_cmp_ft_encoder_l34.pth"
image_dir = "../panos/cropped_panos"
output_dir = "../inference/results_model_v6/images"
results_dir = "../inference/results_model_v6/results.csv"
os.makedirs(output_dir, exist_ok=True)
num_classes = 4
ignore_class = 0
image_size = (512, 1024)

#Transform
transform = T.Compose([
    T.Resize(image_size),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

#Load model
model = smp.DeepLabV3Plus(
    encoder_name="resnet101",       
    encoder_weights="imagenet",  
    in_channels=3,                  
    classes=num_classes         
)
model.load_state_dict(torch.load(model_path, map_location=device)) #Load best model (V6)
model.to(device)
model.eval()

#Process all Mapillary images
results = []
all_filenames = sorted(os.listdir(image_dir))

for filename in all_filenames:
    #Read image
    img_path = os.path.join(image_dir, filename)
    img = Image.open(img_path).convert("RGB")
    input_tensor = transform(img).unsqueeze(0).to(device)

    #Run inference
    with torch.no_grad():
        output = model(input_tensor)
        pred = torch.argmax(output.squeeze(), dim=0).cpu().numpy()

    #Extract IDs
    base = os.path.splitext(filename)[0]
    edge_id, pano_id = base.split("_", 1)

    #Calculate percentage of pixels per class, ignore class 0
    total_pixels = (pred != ignore_class).sum()
    percentages = []
    for cls in range(1, num_classes):
        class_pixels = (pred == cls).sum()
        pct = (class_pixels / total_pixels * 100) if total_pixels > 0 else 0.0
        percentages.append(round(pct, 4))

    #Save image with masks overlays
    if filename in all_filenames:

        #Define colors for each mask class
        colors = {
            0: (0, 0, 0, 0),          # transparent for ignore_class
            1: (0, 0, 255, 100),      # semi-transparent blue for background
            2: (0, 255, 0, 100),      # semi-transparent green for porosity
            3: (255, 0, 0, 100)       # semi-transparent red for wall
        }

        # Resize prediction to match original size
        pred_resized = Image.fromarray(pred.astype(np.uint8)).resize(img.size, resample=Image.NEAREST)
        pred_np = np.array(pred_resized)

        # Create empty RGBA image for mask
        mask_rgba = Image.new("RGBA", img.size, (0, 0, 0, 0))
        mask_pixels = mask_rgba.load()

        # Assign colors per pixel according to class
        for y in range(pred_np.shape[0]):
            for x in range(pred_np.shape[1]):
                cls = pred_np[y, x]
                mask_pixels[x, y] = colors.get(cls, (0, 0, 0, 0))

        # Convert original image to RGBA
        img_rgba = img.convert("RGBA")

        # Overlay mask with transparency
        overlayed = Image.alpha_composite(img_rgba, mask_rgba)

        # Save overlayed image
        overlayed.save(os.path.join(output_dir, f"{base}_overlay.png"))

    # Add to results
    results.append({
        "edge_id": edge_id,
        "pano_id": pano_id,
        "pct_background": percentages[0], # Class 1
        "pct_porosity": percentages[1],   # Class 2
        "pct_wall": percentages[2],       # Class 3
    })

# Create final dataframe
df = pd.DataFrame(results)

df.to_csv(results_dir, index=False)
print("'results.csv' saved")

'results.csv' saved
